In [1]:
import requests
import json
import pandas as pd


In [2]:
url ="http://publicplansdata.org/api/"

In [3]:
params = {
    "q": "ListDataSets",
     "format": "json"
    
}


In [4]:
response = requests.get(url, params = params)
print(response.url)

http://publicplansdata.org/api/?q=ListDataSets&format=json


In [5]:
data = response.json()
data

[{'status': 'OK',
  'date': '2021-05-27 03:10:26',
  'q': 'listdatasets',
  'params': 'q=ListDataSets&format=json',
  'recordcount': 34},
 {'id': '3',
  'table_name': 'pensiongasbschedules',
  'table_display_name': 'Funding'},
 {'id': '4',
  'table_name': 'pensiongasbassumptions',
  'table_display_name': 'Assumptions and Methods'},
 {'id': '5',
  'table_name': 'pensionincomestatement',
  'table_display_name': 'Income Statement'},
 {'id': '7',
  'table_name': 'pensioninvestmentreturn',
  'table_display_name': 'Investment Returns'},
 {'id': '8',
  'table_name': 'pensionactuarialcosts',
  'table_display_name': 'Actuarial Costs'},
 {'id': '9',
  'table_name': 'pensionprovisions',
  'table_display_name': 'Plan Provisions'},
 {'id': '10',
  'table_name': 'pensionmembership',
  'table_display_name': 'Membership'},
 {'id': '11',
  'table_name': 'pensionreportingdates',
  'table_display_name': 'Plan Reporting'},
 {'id': '12',
  'table_name': 'pensioneegroupbasics',
  'table_display_name': 'Empl

In [6]:
params2 = {
    "q": "pensionsystemdata",
     "format": "json",
    #"variables": 
     #"v": variables,
}


In [7]:
response = requests.get(url, params = params2)
print(response.url)

http://publicplansdata.org/api/?q=pensionsystemdata&format=json


In [8]:
data = response.json()
data

[{'status': 'ERROR',
  'date': '2021-05-27 03:10:26',
  'q': 'pensionsystemdata',
  'params': 'q=pensionsystemdata&format=json',
  'recordcount': 1},
 {'Error': "Unknown query type 'pensionsystemdata'"}]

In [9]:
params3 = {
    "q": "QDataSet",
    "dataset": "pensionsystemdata",
     "format": "json",
    #"variables": 
     #"v": variables,
}


In [10]:
response = requests.get(url, params = params3)
print(response.url)

http://publicplansdata.org/api/?q=QDataSet&dataset=pensionsystemdata&format=json


In [11]:
data = response.json()
data

[{'status': 'OK',
  'date': '2021-05-27 03:10:27',
  'q': 'qdataset',
  'params': 'q=QDataSet&dataset=pensionsystemdata&format=json',
  'recordcount': 3431},
 {'fy': '2001',
  'system_id': '1',
  'sys_actives_tot': '78019',
  'sys_adjustment_MktAssets': None,
  'sys_Beneficiaries_other': None,
  'sys_Beneficiaries_survivors': None,
  'sys_beneficiaries_tot': '26653',
  'sys_contrib_EE_other': None,
  'sys_contrib_EE_PurchaseService': None,
  'sys_contrib_EE_regular': '133080.000',
  'sys_contrib_ER_other': None,
  'sys_contrib_ER_regular': '130081.000',
  'sys_contrib_ER_state': None,
  'sys_contrib_other': '1370.000',
  'sys_contrib_tot': '264531.000',
  'sys_DROPMembers': None,
  'sys_expense_AdminExpenses': '-4850.000',
  'sys_expense_alternatives': None,
  'sys_expense_COLABenefits': None,
  'sys_expense_DeathBenefits': '-24985.000',
  'sys_expense_Depreciation': '-330.000',
  'sys_expense_DisabilityBenefits': None,
  'sys_expense_DROPBenefits': None,
  'sys_expense_investments': '

In [12]:
data_df = pd.DataFrame(data[1:])


In [13]:
data_df.iloc[0:10,0:5]

,fy,system_id,sys_actives_tot,sys_adjustment_MktAssets,sys_Beneficiaries_other
0,2001,1,78019,None,None
1,2002,1,80582,None,None
2,2003,1,81876,None,None
3,2004,1,82642,None,None
4,2005,1,81581,None,None
5,2006,1,83170,None,None
6,2007,1,84814,None,None
7,2008,1,87016,None,None
8,2009,1,88338,None,None
9,2010,1,87989,None,None


In [16]:
data_df.head()

,fy,system_id,sys_actives_tot,sys_adjustment_MktAssets,sys_Beneficiaries_other,sys_Beneficiaries_survivors,sys_beneficiaries_tot,sys_contrib_EE_other,sys_contrib_EE_PurchaseService,sys_contrib_EE_regular,...,sys_income_PrivateEquity,sys_income_RealEstate,sys_income_SecuritiesLending,sys_MktAssets_net,sys_NumberofEmployers,sys_OtherMembers,sys_SecLendRebate,sys_ServiceRetirees,sys_SpousalSurvivors,sys_TotMembership
0,2001,1,78019,None,None,None,26653,None,None,133080.000,...,None,None,None,7459904.000,1141,None,-19439.000,None,None,111705
1,2002,1,80582,None,None,None,27335,None,None,151401.000,...,None,None,None,6617744.000,1141,None,-4169.000,None,None,115752
2,2003,1,81876,None,None,None,25973,None,None,162105.000,...,None,None,None,7429483.000,1141,None,-2181.000,None,None,119566
3,2004,1,82642,None,None,None,29227,None,None,169343.000,...,None,None,None,8017311.000,1141,None,-3401.000,None,None,122282
4,2005,1,81581,None,None,None,30270,None,None,160478.000,...,None,None,None,8697927.000,1141,None,-15310.000,None,None,122843
